# Z_test to reduce features
- attempt to reduce the number of features considered by the neural networks by performing a two-sided Z-test with hypotheses:
    - H_0: mean_AD = mean_ND
    - H_a: mean_AD =/= mean_ND

In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

from statistics import mean, variance, stdev
from scipy.stats import norm


In [2]:
expression_df = pd.read_excel("GSE132903_Matrix_Normalized.xlsx")

In [3]:
# Standardize after removing 1st two columns (probe names and probe IDs which are strings)
std_expressions = StandardScaler().fit_transform(( expression_df.iloc[:,2:]))
# Get the probe names for rows, and sample IDs for columns
probes = expression_df["Illumina probe name"]
sample_ids = expression_df.columns[2:]
# Restructure numpy array into a data frame
std_expr_df = pd.DataFrame(columns=sample_ids, data=std_expressions, index=probes)
std_expr_df

,ND_1_08-81,ND_2_08-85,ND_4_08-72,ND_5_08-83,ND_6_97-53,ND_7_98-19,ND_8_97-46,ND_9_97-17,ND_10_97-19,ND_11_97-45,...,ND_14_97-09,ND_20_99-29,ND_15_97-10,ND_21_99-22,ND_16_97-02,ND_22_99-02,ND_17_97-14,ND_23_98-22,ND_18_97-37,ND_24_98-32
Illumina probe name,,,,,,,,,,,,,,,,,,,,,
ILMN_1762337,-0.534481,-0.564305,-0.677523,-0.625994,-0.546681,-0.586291,-0.609140,-0.552293,-0.600437,-0.622244,...,-0.610331,-0.626355,-0.659488,-0.493454,-0.609828,-0.566150,-0.607800,-0.638868,-0.604595,-0.617487
ILMN_2055271,-0.229969,0.720740,-0.150065,-0.381419,-0.502166,-0.316202,-0.348732,-0.341852,-0.422897,-0.372415,...,-0.524308,-0.509714,-0.444066,-0.381570,-0.317405,-0.493416,-0.310092,-0.467656,-0.487376,-0.446404
ILMN_1736007,-0.544630,-0.691425,-0.546998,-0.559934,-0.566883,-0.590596,-0.545981,-0.525486,-0.601876,-0.560318,...,-0.510969,-0.565312,-0.568123,-0.638987,-0.572609,-0.565095,-0.582114,-0.576455,-0.562028,-0.536636
ILMN_2383229,-0.699938,-0.657741,-0.691226,-0.558644,-0.674641,-0.619386,-0.669988,-0.595791,-0.664792,-0.629676,...,-0.644097,-0.649979,-0.620379,-0.612786,-0.588619,-0.687507,-0.669112,-0.616731,-0.656940,-0.654039
ILMN_1806310,-0.569721,-0.443287,-0.490081,-0.552808,-0.512096,-0.493838,-0.451184,-0.499022,-0.550157,-0.535868,...,-0.617304,-0.568477,-0.470841,-0.568290,-0.586733,-0.576187,-0.498237,-0.631950,-0.553921,-0.543424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ILMN_2371169,0.214081,0.348751,0.332673,0.138926,0.084933,0.347371,0.098588,0.166828,0.130057,0.129717,...,0.561677,0.711658,0.908740,0.341084,0.671970,0.598353,0.635920,0.576570,0.555201,0.443776
ILMN_1701875,1.721395,1.718369,1.865121,1.458299,1.401641,1.971051,1.543323,1.541852,1.327174,1.381246,...,1.390366,1.461087,1.636568,1.283997,1.529294,1.304456,1.360008,1.566179,1.190420,1.168796
ILMN_1786396,1.560826,1.302309,1.466605,1.376396,1.239233,1.470302,1.172215,1.573805,1.335072,1.250824,...,1.459148,1.364479,1.555088,1.482821,1.349149,1.153456,1.430945,1.504348,1.235779,1.250699


In [4]:
# Split standardized DF by AD and ND sample status
ad_cols = [sid for sid in sample_ids if "AD" in sid]
nd_cols = [sid for sid in sample_ids if "ND" in sid]
AD_std_expr = std_expr_df.loc[:,ad_cols]
ND_std_expr = std_expr_df.loc[:,nd_cols]
AD_std_expr

,AD_96_02-41,AD_97_03-04,AD_98_02-28,AD_99_03-18,AD_100_03-21,AD_101_03-27,AD_102_01-45,AD_103_01-48,AD_104_01-53,AD_105_08-45,...,AD_197_00-47,AD_198_01-13,AD_199_98-18,AD_176_96-04,AD_177_97-39,AD_178_97-48,AD_179_98-07,AD_180_96-21,AD_181_98-10,AD_182_95-31
Illumina probe name,,,,,,,,,,,,,,,,,,,,,
ILMN_1762337,-0.599463,-0.605132,-0.535729,-0.545943,-0.574224,-0.536027,-0.596378,-0.555872,-0.528082,-0.571020,...,-0.566044,-0.626611,-0.638534,-0.634750,-0.561810,-0.585452,-0.641195,-0.519704,-0.572054,-0.643878
ILMN_2055271,-0.337900,-0.307214,-0.270125,-0.268590,-0.364186,-0.471851,-0.259883,-0.335354,-0.179661,-0.393364,...,-0.505629,0.004400,-0.399698,-0.411206,-0.460120,-0.492055,-0.391048,-0.341823,-0.461496,-0.415323
ILMN_1736007,-0.530327,-0.605132,-0.607202,-0.532975,-0.624816,-0.525193,-0.612610,-0.532837,-0.594888,-0.564031,...,-0.670440,-0.547708,-0.505996,-0.625831,-0.599168,-0.502778,-0.500940,-0.569203,-0.605541,-0.570111
ILMN_2383229,-0.623974,-0.596866,-0.562107,-0.556334,-0.672037,-0.667239,-0.541058,-0.586939,-0.723610,-0.632109,...,-0.653981,-0.632741,-0.574072,-0.719772,-0.645047,-0.673295,-0.587265,-0.539113,-0.677982,-0.655130
ILMN_1806310,-0.471384,-0.524983,-0.461149,-0.516243,-0.493385,-0.547191,-0.502242,-0.530495,-0.437741,-0.558354,...,-0.641166,-0.590961,-0.500635,-0.506873,-0.487395,-0.463637,-0.516540,-0.394106,-0.520607,-0.580609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ILMN_2371169,-0.036567,-0.149770,0.020755,0.045002,-0.071008,-0.008239,0.053235,-0.095905,0.186415,0.006885,...,0.155161,0.403996,-0.139924,0.341242,-0.082026,0.081018,-0.036826,0.226933,0.092589,0.175543
ILMN_1701875,1.516043,1.305309,1.607321,1.343023,1.208351,1.682157,1.439425,1.462909,1.499938,1.529970,...,1.736845,1.861313,1.408217,1.601191,1.393127,1.325069,1.014410,1.827547,1.544670,1.643924
ILMN_1786396,1.526577,1.516522,1.445553,1.432791,1.536276,1.663656,1.580840,1.757802,1.584440,1.374438,...,1.533771,1.486283,1.037421,1.275973,1.361775,1.364013,1.088910,1.259187,1.385451,1.527203


In [5]:
# Perform Z-test for each probe by WRT means of AD and ND samples. 
# Iterate through various significant thresholds to see how many probes hit for significance 
n_ad = AD_std_expr.shape[1]
n_nd = ND_std_expr.shape[1]

def two_sided_tail_pval(Z_stat):
    return 2 * (1 - norm.cdf(abs(Z_stat)))

# Performing the tests takes some time. I added a simple progress monitor
p_vals = []
for i, probe in enumerate(probes):
    if not bool(i % 5000):
        print(f"{i / len(probes):2.0%} complete")
    ad_mean_expr = mean(AD_std_expr.loc[probe])
    nd_mean_expr = mean(ND_std_expr.loc[probe])
    ad_var = variance(AD_std_expr.loc[probe])
    nd_var = variance(ND_std_expr.loc[probe])
    # Testing H_0: mean_AD = mean_ND, H_a: mean_AD =/= mean_ND
    Z = (ad_mean_expr - nd_mean_expr) / pow(ad_var/n_ad + nd_var/n_nd, 0.5)
    p_vals.append(two_sided_tail_pval(Z))
print("Finished")



0% complete
12% complete
24% complete
36% complete
47% complete
59% complete
71% complete
83% complete
95% complete
Finished


In [6]:
alphas = [pow(10,-1 * i) for i in range(6,16)]

for alpha in alphas:
    significance_series = pd.Series( index=std_expr_df.index, data=[p < alpha for p in p_vals])
    sig_number = std_expr_df.loc[significance_series].shape[0]
    print(f"alpha = {alpha:2.1e} found: {sig_number} significantly expressed genes.")


alpha = 1.0e-06 found: 5765 significantly expressed genes.
alpha = 1.0e-07 found: 4260 significantly expressed genes.
alpha = 1.0e-08 found: 3200 significantly expressed genes.
alpha = 1.0e-09 found: 2322 significantly expressed genes.
alpha = 1.0e-10 found: 1708 significantly expressed genes.
alpha = 1.0e-11 found: 1208 significantly expressed genes.
alpha = 1.0e-12 found: 877 significantly expressed genes.
alpha = 1.0e-13 found: 612 significantly expressed genes.
alpha = 1.0e-14 found: 400 significantly expressed genes.
alpha = 1.0e-15 found: 266 significantly expressed genes.


In [31]:
# Write the significance filtered expression dataframes to csv for alphas 10^-12, 10^-14, 10^-15
for alpha in alphas[6:]:
    significance_series = pd.Series( index=std_expr_df.index, data=[p < alpha for p in p_vals])
    sig_std_expr = std_expr_df.loc[significance_series].transpose()
    num_genes = sig_std_expr.shape[1]
    # Concatonate a series w/ targets to the differentially expressed dataframes. 1 for AD sample, 0 for ND
    targets = [int("AD" in samp_id) for samp_id in  sig_std_expr.index]
    targets_series = pd.Series(data=targets, index=sig_std_expr.index, name="target")
    sig_std_expr = pd.concat([sig_std_expr, targets_series], axis=1)
    sig_std_expr.to_csv(f"{num_genes}_most_differentially_expressed_probes.csv")
